In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as sa
import mysql

In [7]:
def get_data(ticker: str, start_date: str=None, end_date: str=None):
    return yf.download(tickers=ticker, start=start_date, end=end_date)

def generate_signal(data):
    signals = pd.DataFrame(data.index)
    signals['signal'] = 0

    # Create a short simple moving average over the short window
    signals['short_mavg'] = data['Close'].rolling(window=40, min_periods=1, center=False).mean()

    # Create a long simple moving average over the long window
    signals['long_mavg'] = data['Close'].rolling(window=100, min_periods=1, center=False).mean()

    # Create signals
    signals['signal'][40:] = np.where(signals['short_mavg'][40:] > signals['long_mavg'][40:], 1.0, 0.0)

    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals

def backtest_strategy(signals, data, initial_capital=10000):
    positions = pd.DataFrame(index=signals.index).fillna(0.0)
    positions['stock'] = 100 * signals['signal']   # Buy 100 shares on each buy signal

    # Initialize the portfolio with value owned
    portfolio = positions.multiply(data['Adj Close'], axis=0)

    # Store the difference in shares owned
    pos_diff = positions.diff()

    # Add 'cash' to portfolio
    portfolio['cash'] = initial_capital - (pos_diff.multiply(data['Adj Close'], axis=0)).cumsum()

    # Add 'total' to portfolio
    portfolio['total'] = portfolio['cash'] + portfolio['stock']

    return portfolio

In [8]:
data = get_data('voo', start_date='2022-01-01', end_date='2024-01-01')
signal = generate_signal(data=data)
portfolio = backtest_strategy(signal, data)

[*********************100%%**********************]  1 of 1 completed
C:\Users\wongty\AppData\Local\Temp\ipykernel_41196\2658335831.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  signals['signal'][40:] = np.where(signals['short_mavg'][4

In [14]:
portfolio.tail(20)

,stock,cash,total
2023-12-01 00:00:00,NaN,NaN,NaN
2023-12-04 00:00:00,NaN,NaN,NaN
2023-12-05 00:00:00,NaN,NaN,NaN
2023-12-06 00:00:00,NaN,NaN,NaN
2023-12-07 00:00:00,NaN,NaN,NaN
2023-12-08 00:00:00,NaN,NaN,NaN
2023-12-11 00:00:00,NaN,NaN,NaN
2023-12-12 00:00:00,NaN,NaN,NaN
2023-12-13 00:00:00,NaN,NaN,NaN
2023-12-14 00:00:00,NaN,NaN,NaN


In [5]:
# functional method
engine = sa.create_engine(url='mysql+mysqldb://@localhost/test', echo=True)
metadata = sa.MetaData()

table_name = ''
table = sa.Table(
    table_name,
    metadata,
    # schema=,
    sa.Column('id', sa.Integer, primary_key=True),
    sa.Column('date', sa.String),
    sa.Column('open', sa.Double),
    sa.Column('high', sa.Double),
    sa.Column('low', sa.Double),
    sa.Column('close', sa.Double)
)

metadata.create_all(engine)

DatetimeIndex(['1980-12-12', '1980-12-15', '1980-12-16', '1980-12-17',
               '1980-12-18', '1980-12-19', '1980-12-22', '1980-12-23',
               '1980-12-24', '1980-12-26',
               ...
               '2024-06-27', '2024-06-28', '2024-07-01', '2024-07-02',
               '2024-07-03', '2024-07-05', '2024-07-08', '2024-07-09',
               '2024-07-10', '2024-07-11'],
              dtype='datetime64[ns]', name='Date', length=10985, freq=None)

In [ ]:
# OOP method
from sqlalchemy.orm import Mapped, mapped_column, sessionmaker, declarative_base

db = sa.create_engine()
Session = sessionmaker(bind=db)
Base = declarative_base()

class Stock(Base):
    __tablename__ = 'stocks'
    id: Mapped[int] = mapped_column(primary_key=True, index=True, unique=True)
    open: Mapped[str]
    high: Mapped[str]
    low: Mapped[float]
    close: Mapped[float]

    def __repr__(self) -> str:
        return f'<Stock(id={self.id}, open={self.open}, high={self.high}, low={self.low}, close={self.close})>'

    def main():
        Base.metadata.create_all(db)
        user = Stock(username='', email='')

        with Session() as session:
            session.add(user)
            session.commit

if __name__ = ''

In [ ]:
connection = engine.connect()
instruments = 'Stocks'
engine.execute(f'CREATE SCHEMA IF NOT EXISTS {instruments}')

def insert_data(date: str, open: float, high: float, low: float, close: float):
    query = table.insert().values(date=date, open=open, high=high, low=low, close=close)
    connection.execute(query)

def select_data(date: str, open: float, high: float, low: float, close: float):
    query = table.select().where(table.)
    result = connection.execute(query)
    return result.fetchall()

def delete_data():
    pass

In [ ]:
stocks.loc["2019-07-22":"2024-07-01"].plot(y='Close')

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host='',
    user='',
    password='',
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE")

In [ ]:
table_name = tickers
sql = f'INSERT INTO {table_name} () VALUES ()'
values = ()
mycursor.execute(sql, values)

mydb.commit()